# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 31 2022 10:25AM,249682,19,ADV80007172,"AdvaGen Pharma, Ltd",Released
1,Oct 31 2022 9:38AM,249681,10,CTF0011363-BO,"Citieffe, Inc.",Released
2,Oct 31 2022 9:38AM,249680,20,ATR-I000016,"HVL, LLC dba Atrium Innovations",Released
3,Oct 31 2022 9:34AM,249679,10,8658551,Eywa Pharma Inc.,Released
4,Oct 31 2022 9:34AM,249679,10,8645697-BO,Eywa Pharma Inc.,Released
5,Oct 31 2022 9:34AM,249679,10,8663669,Eywa Pharma Inc.,Released
6,Oct 31 2022 9:34AM,249679,10,8663670,Eywa Pharma Inc.,Released
7,Oct 31 2022 9:34AM,249679,10,8662463-BO,Eywa Pharma Inc.,Released
8,Oct 31 2022 9:34AM,249679,10,8662465-BO,Eywa Pharma Inc.,Released
9,Oct 31 2022 9:34AM,249679,10,8664221,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,249678,Released,1
15,249679,Released,31
16,249680,Released,1
17,249681,Released,1
18,249682,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
249678,NaN,1.0
249679,NaN,31.0
249680,NaN,1.0
249681,NaN,1.0
249682,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
249646,8.0,1.0
249651,8.0,10.0
249660,27.0,19.0
249667,12.0,8.0
249670,0.0,49.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
249646,8,1
249651,8,10
249660,27,19
249667,12,8
249670,0,49


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,249646,8,1
1,249651,8,10
2,249660,27,19
3,249667,12,8
4,249670,0,49


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,249646,8,1
1,249651,8,10
2,249660,27,19
3,249667,12,8
4,249670,,49


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 31 2022 10:25AM,249682,19,"AdvaGen Pharma, Ltd"
1,Oct 31 2022 9:38AM,249681,10,"Citieffe, Inc."
2,Oct 31 2022 9:38AM,249680,20,"HVL, LLC dba Atrium Innovations"
3,Oct 31 2022 9:34AM,249679,10,Eywa Pharma Inc.
34,Oct 31 2022 9:12AM,249678,10,Acute Outpatient Solutions
35,Oct 31 2022 9:10AM,249676,10,"Methapharm, Inc."
68,Oct 31 2022 9:09AM,249674,10,"Methapharm, Inc."
78,Oct 31 2022 9:08AM,249675,20,"Exact-Rx, Inc."
81,Oct 31 2022 9:07AM,249673,10,ISDIN Corporation
114,Oct 31 2022 9:04AM,249671,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 31 2022 10:25AM,249682,19,"AdvaGen Pharma, Ltd",,1
1,Oct 31 2022 9:38AM,249681,10,"Citieffe, Inc.",,1
2,Oct 31 2022 9:38AM,249680,20,"HVL, LLC dba Atrium Innovations",,1
3,Oct 31 2022 9:34AM,249679,10,Eywa Pharma Inc.,,31
4,Oct 31 2022 9:12AM,249678,10,Acute Outpatient Solutions,,1
5,Oct 31 2022 9:10AM,249676,10,"Methapharm, Inc.",,33
6,Oct 31 2022 9:09AM,249674,10,"Methapharm, Inc.",,10
7,Oct 31 2022 9:08AM,249675,20,"Exact-Rx, Inc.",,3
8,Oct 31 2022 9:07AM,249673,10,ISDIN Corporation,,33
9,Oct 31 2022 9:04AM,249671,10,ISDIN Corporation,,63


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 10:25AM,249682,19,"AdvaGen Pharma, Ltd",1,
1,Oct 31 2022 9:38AM,249681,10,"Citieffe, Inc.",1,
2,Oct 31 2022 9:38AM,249680,20,"HVL, LLC dba Atrium Innovations",1,
3,Oct 31 2022 9:34AM,249679,10,Eywa Pharma Inc.,31,
4,Oct 31 2022 9:12AM,249678,10,Acute Outpatient Solutions,1,
5,Oct 31 2022 9:10AM,249676,10,"Methapharm, Inc.",33,
6,Oct 31 2022 9:09AM,249674,10,"Methapharm, Inc.",10,
7,Oct 31 2022 9:08AM,249675,20,"Exact-Rx, Inc.",3,
8,Oct 31 2022 9:07AM,249673,10,ISDIN Corporation,33,
9,Oct 31 2022 9:04AM,249671,10,ISDIN Corporation,63,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 10:25AM,249682,19,"AdvaGen Pharma, Ltd",1,
1,Oct 31 2022 9:38AM,249681,10,"Citieffe, Inc.",1,
2,Oct 31 2022 9:38AM,249680,20,"HVL, LLC dba Atrium Innovations",1,
3,Oct 31 2022 9:34AM,249679,10,Eywa Pharma Inc.,31,
4,Oct 31 2022 9:12AM,249678,10,Acute Outpatient Solutions,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 10:25AM,249682,19,"AdvaGen Pharma, Ltd",1,NaN
1,Oct 31 2022 9:38AM,249681,10,"Citieffe, Inc.",1,NaN
2,Oct 31 2022 9:38AM,249680,20,"HVL, LLC dba Atrium Innovations",1,NaN
3,Oct 31 2022 9:34AM,249679,10,Eywa Pharma Inc.,31,NaN
4,Oct 31 2022 9:12AM,249678,10,Acute Outpatient Solutions,1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 31 2022 10:25AM,249682,19,"AdvaGen Pharma, Ltd",1,0.0
1,Oct 31 2022 9:38AM,249681,10,"Citieffe, Inc.",1,0.0
2,Oct 31 2022 9:38AM,249680,20,"HVL, LLC dba Atrium Innovations",1,0.0
3,Oct 31 2022 9:34AM,249679,10,Eywa Pharma Inc.,31,0.0
4,Oct 31 2022 9:12AM,249678,10,Acute Outpatient Solutions,1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2746380,258,47.0
19,499328,2,8.0
20,499355,4,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2746380,258,47.0
1,19,499328,2,8.0
2,20,499355,4,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,258,47.0
1,19,2,8.0
2,20,4,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,258.0
1,19,Released,2.0
2,20,Released,4.0
3,10,Executing,47.0
4,19,Executing,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19,20
Status,,,
Executing,47.0,8.0,0.0
Released,258.0,2.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19,20
0,Executing,47.0,8.0,0.0
1,Released,258.0,2.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19,20
0,Executing,47.0,8.0,0.0
1,Released,258.0,2.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()